In [ ]:
from IPython.display import HTML
from IPython.display import display

# Taken from https://stackoverflow.com/questions/31517194/how-to-hide-one-specific-cell-input-or-output-in-ipython-notebook
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.cell.code_cell.rendered.selected div.input').hide();
    } else {
        $('div.cell.code_cell.rendered.selected div.input').show();
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Some hidden code... to show/hide this cell's raw code input, click (or double click if stubborn!) <a href="javascript:code_toggle()">here</a>.''')
display(tag)

############### Write code below ##################
from IPython.core.display import HTML
def css_styling():
    styles = open("./STUFF/colors.css", "r").read()
    return HTML(styles)
css_styling()

<div class=warn>
These blue boxes contain instructions for you to follow, or stuff for you to do
<h2>How to access this Jupyter notebook</h2>

* <b>Step 1</b>: Open a web browser, go to [this page](https://warwick.ac.uk/fac/sci/chemistry/research/maurer/maurergroup/software/iatl_strategic_project/), scroll down, download the Login App (the Windows exacutable if using ITS machines) and double click on the executable (usually ended up into the Download folder) <br>
* <b>Step 2</b>: The Notebook Launcher pops up: select the CH274 module and fill in the boxes using your SCRTP username and password <br>
* <b>Step 3</b>: Open the Jupyter notebook you are interested in, e.g. KS_3_Class.ipynb <br>
* <b>Step 4</b>: Make a copy of the orginal notebook (which is read only). In the toolbar on top of the notebook: File - Make a Copy <br>
* <b>Step 5</b>: You're all set! <br><br>
<b> Remember: </b> You can access this notebook at home at any time by going through the same steps on e.g. your laptop - all the changes you have made will be saved and synced!<br>
<div/>

# CH274: Computational Workshop 3
## Using a Hückel MO code to study Polyaromatic Hydrocarbons (PAHs)

In this workshop, we are going to
* see how a Huckel/tight-binding Python code works
* apply this code to study the stability and reactivity of three different polyaromatic hydrocarbons, namely:
<img src="STUFF/3mols.png">

These three molecules are PAH molecules, which are created from incomplete combustion in cars, industrial exhaust, and soot. They are known potent carcinogens. All three have the same number of sp2 hybridised carbon atoms (22) and we can compare them directly with Hückel theory.





Based on the contents of this workshop, you will be given a homework assignment, which you will be able to access in the Assignments Tab.


## Part 1: Setting up the input data

We will first set up our input data for the three molecules. The code will require the respective Hückel matrices as input. 
A Hückel matrix is basicallz nothing else than a connectivity matrix. The connectivity matrix tells us which carbon atoms are connected to each other. If atoms are bonded, the matrix element is -1, if they are non-bonded, the matrix element has a 0.

The following cell reads the required information for the three molecules from a database of more than 500 PAHs and then prints the connectivity matrices and the Lewis formulas.

In [ ]:
from rdkit import Chem #RDKIT is a useful package for drawing molecules and operating on them
import rdkit.Chem.Draw
from rdkit.Chem import AllChem
from IPython.display import SVG, display
#functions that operate on the PAH database
from PAH_data import select_molecule, matprint, mol_with_atom_index
import matplotlib.pyplot as plt

#######################
#list of molecule names
names = [
    'Indeno[1,2,3-cd]fluoranthene',
    'Dibenzo[cd,jk]pyrene',
    'Indeno[1,2,3-cd]pyrene',
]
molecules = []
rdkit_objects = []
#this selects the molecules from the database of 500 PAHs, transforms it into an rdkit 'molecule object'
for i in range(3):
    mol, mol_object = select_molecule(names[i])
    molecules.append(mol)
    rdkit_objects.append(mol_object)

#These rdkit objects are stored in the 'molecules' list. 
#Together with other thigns, they contain the connectivity matrices

#list of three connnectivity matrices
conmats = [(molecules[0])['conmat'], molecules[1]['conmat'], molecules[2]['conmat']]
#list of three molecule names
names = names
#list of three RDKIT molecule objects
molecules = rdkit_objects

#The RDKIT molecule objects can be used for visualization
#display without atom numbering
display(SVG(Chem.Draw.MolsToGridImage([molecules[0]], useSVG=True, legends=[names[0]], subImgSize=(300,300))))
#display with numbering
display(SVG(Chem.Draw.MolsToGridImage([mol_with_atom_index(molecules[0])], useSVG=True, legends=[names[0]], subImgSize=(300,300))))
#print the connectivity matrix for molecule 1
matprint(conmats[0])

#molecule 2
display(SVG(Chem.Draw.MolsToGridImage([mol_with_atom_index(molecules[1])], useSVG=True, legends=[names[1]], subImgSize=(300,300))))
matprint(conmats[1])

#molecule 3
display(SVG(Chem.Draw.MolsToGridImage([mol_with_atom_index(molecules[2])], useSVG=True, legends=[names[2]], subImgSize=(300,300))))
matprint(conmats[2])
#This line generates the picture of the three molecules



<div class=warn>

<b> Task 1: Use the following text cell to make notes and answer following questions:</b><br>

<ul>
<li> Verify that the connectivity matrices are correct and that they are consistent with the provided atom numbering.</li>
<li> Are the three molecules alternant or non-alternant? </li>

</ul>

</div>

#### Answers to Task 1:

INSERT YOUR ANSWER HERE



We can consider the connectivity matrices to be Hückel matrices, where we have set
$$\alpha=\langle \phi_i|H|\phi_i\rangle = 0 $$
and 
$$\beta=\langle \phi_i|H|\phi_j\rangle = -1 $$

**NOTE**: Because we have set $\beta=-1$, our energies will be numbers that already reflect the correct energy ordering. Therefore, we do not have to consider the implicit fact that $\beta<0$ in $2\beta$ ie. the lower the number, the more stable the energy level. We have to use actual numbers, as we cannot use variables $\alpha,\beta$ in a numerical code.

## Part 2: How to use the Hückel code


The code we will use is called ```shmo``` and is documented here:
https://github.com/randlet/SHMO

Let's learn how to use the code with the example of Acenaphthalene.
We first pull the Hückel matrix of Acenaphthalene from the database and then initialise the HuckelSolver with it.

<div class=warn>

<b> Task 2:</b> Explore the Hückel SHMO module<br>

</div>

In [ ]:
import numpy as np
import shmo

In [ ]:
#select Acenaphthalene
mol, rdkit_mol = select_molecule('Acenaphthalene')

#get hueckel matrix
huckel_matrix = mol['conmat']

#draw molecule from rdkit mol object
display(SVG(Chem.Draw.MolsToGridImage([rdkit_mol], useSVG=True, legends=['Acenaphthalene'], subImgSize=(160,160))))

#print hueckel matrix
matprint(huckel_matrix)

In [ ]:
#initialise hueckel solver
solver = shmo.HuckelSolver(data=huckel_matrix)
##NOTE!: Each time you change the molecule, you need to reinitialise the Huckel solver, 
#ie. call the above line with a new huckel matrix!

Now that the Hückel solver is initialised, we can calculate a lot of different properties.
If you want to find out about all the different properties you can calculate, try the following:

Use the next code cell to type:
```
solver.
```


and then press the tab key. This will give you a drop-down list of all possible functions you can access with

```
solver.<functionname>
```

As usual, placing a question mark behind the function, will tell you what it does.

In [ ]:
#YOUR CODE HERE
#


In [ ]:
#Let's print the MO energies
print(solver.energies)

#energy level diagram for benzene
plt.plot(np.zeros(len(solver.energies)), solver.energies,lw=0.0,marker='_',ms=25,mew=3)
plt.xlim(-2,2)
plt.ylabel('energies')
plt.show()

In [ ]:
#MO eigenvectors
eigenvecs= solver.eigen_vectors
#every column is one coefficient vector
matprint(np.array(eigenvecs))

For molecules of this size, there is no way we can visually inspect the coefficients and understand the molecular orbitals. Below, you will find a function that allows you to visualise the orbitals.

In [ ]:
%matplotlib inline 
from ipywidgets import interact, interactive, fixed, interact_manual
from PAH_data import draw_MO

interactive(draw_MO, mol=fixed(rdkit_mol), eigenvecs=fixed(eigenvecs), n=range(len(eigenvecs)))


We can also visualize the number of electrons on each carbon atom

In [ ]:
#charge populations
print(solver.charge_densities)
draw_MO(rdkit_mol, solver.charge_densities)

We can visualize the net charges on each atom

In [ ]:
#net charges (charge populations - atomic charge)
print(solver.net_charges)
draw_MO(rdkit_mol, solver.net_charges*2) #I added a scaling factor of 2 to make the net charges more visible

We can visualize the self-polarisability of each atom

In [ ]:
#atomic self-polarisabilities
self_polarisabilities = np.diag(solver.aa_polar)
print(self_polarisabilities)
draw_MO(rdkit_mol, self_polarisabilities)

We can also calculate the **total energy** and the **bond order**:

In [ ]:
#total energy
def calculate_total_energy(solver):
    """
    This function calculates the total energy
    """
    total_energy = 0.0
    for state in solver.populated_levels:
        total_energy += state[0]*state[2]
    return total_energy

#neutral molecule energy
print('neutral molecule')
solver.set_num_electrons(12)
print(calculate_total_energy(solver))

n = calculate_total_energy(solver)

nelectrons = solver.num_electrons

#make cation
print('cation')
solver.set_num_electrons(nelectrons-1)
print(calculate_total_energy(solver))

c = calculate_total_energy(solver)

#make anion
print('anion')
solver.set_num_electrons(nelectrons+1)
print(calculate_total_energy(solver))

solver.set_num_electrons(nelectrons)

In [ ]:
def print_bond_orders(mat, solver):
    """
    print bond orders to screen
    """

    bond_orders=np.multiply(-mat,solver.bond_orders)

    print('atom i    atom j    bond order')
    atom_pairs = []
    borders = []
    for pair in solver.bond_pairs():
        i, j = pair
        print("  {0:3d}      {1:3d}      {2:8.3f}".format(i+1, j+1, bond_orders[i,j]))
        atom_pairs.append([i+1,j+1])
        borders.append(bond_orders[i,j])

    return atom_pairs, borders
        
atom_pairs, bond_orders = print_bond_orders(huckel_matrix, solver)

## Part 3: Hückel calculation and analysis of the three molecules

<div class=warn>

<b> Task 4</b><br>

<ol>
<li> Calculate the total energies of Indeno[1,2,3-cd]fluoranthene, Dibenzo[cd,jk]pyrene, Indeno[1,2,3-cd]pyrene. Which molecule is more stable? </li>
<li> Calculate the cations and anions of the three molecules. Which cation and which anion is more stable? Use the energy levels and wave functions to explain the stability trend.</li>
<li> For the three molecules, find out which carbon atom is most easily attacked by (A) a nucleophililic reaction agent and (B) an electrophilic reaction agent.</li>

</ol>

</div>